In [16]:
# -*- coding:utf-8 -*-

# general imports
import os.path as op
import os

# third party imports
import numpy as np
import nibabel as nib
from scipy.ndimage.interpolation import zoom
import dicom2nifti
import SimpleITK as sitk
import matplotlib.image as mpimg

In [17]:
require_labels = [0,    # background
                  1,    # spleen
                  2,    # right kidney
                  3,    # left kidney
                  6]    # liver

resample_size = [256,256,32]
path = ['/root/autodl-tmp/Train_Sets/MR/']
save_path = '../../Dataset/CHAOS/CHAOS_%d_%d_%d/' % \
            (resample_size[0], resample_size[1], resample_size[2])

In [18]:
def create_path(pathlist):
    for path in pathlist:
        if not op.exists(path):
            os.makedirs(path)

def resize_label(img, label, spacing, resample_size):
    assert img.shape == label.shape
    origin_shape = np.array(img.shape)
    new_space = spacing * origin_shape / resample_size
    resize_factor = 1.0 * np.asarray(resample_size) / np.asarray(origin_shape)
    img = zoom(img, resize_factor, order=1) #order = 1：bilinear interpolaion
    label = zoom(label, resize_factor, order=0) #order = 0:nearest
    return img, label, new_space

In [22]:
for j in range(1):
    image_path = path[j]
    files = os.listdir(image_path)
    files.sort()
    print(files,len(files))
    reader = sitk.ImageSeriesReader()
    for patient in files:

        img_names = reader.GetGDCMSeriesFileNames(image_path + patient +'/T2SPIR/DICOM_anon')
        reader.SetFileNames(img_names)
        image = reader.Execute()
        array = sitk.GetArrayFromImage(image).transpose(1,2,0) 
        spacing = image.GetSpacing()#(x,y,z)
        print(array.shape,spacing)
        print(array.dtype)
        nii_folder = '../../Dataset/CHAOS/CHAOS_nii/'+str(patient)
        if not os.path.exists(nii_folder):
            os.mkdir(nii_folder)
        nii_file = dicom2nifti.dicom_series_to_nifti(image_path + patient +'/T2SPIR/DICOM_anon', os.path.join(nii_folder, "procimg.nii.gz"))
        
        if j == 0:
            labels = os.listdir(image_path + patient +'/T2SPIR/Ground')
            labels.sort()
            print(labels)
            #get the mask and concat them
            for i in range(len(labels)):
                if i == 0:
                    label = mpimg.imread(image_path + patient +'/T2SPIR/Ground/'+labels[i])[...,np.newaxis]
                else:
                    mask = mpimg.imread(image_path + patient +'/T2SPIR/Ground/'+labels[i])[...,np.newaxis]
                    label = np.concatenate((label,mask),axis=2)

            print('label',label.shape)
            #in mask .png the label with different number , we should change it
            label[label == 0.9882353] = 1
            label[label == 0.49411765] = 2
            label[label == 0.7411765] = 3
            label[label == 0.24705882] = 6
            label[label == 0] = 0

            label = label.astype(np.int8)
            print(label.dtype)
            
            label_unique = np.unique(label)
            print('label_unique',label_unique)
            new_array, new_label, new_spacing = resize_label(array, label, spacing, resample_size)
            new_array = new_array.astype(np.float32)
            print(patient,new_array.shape,new_label.shape,new_spacing)
        
        else :
            origin_shape = np.array(array.shape)
            new_space = spacing * origin_shape / resample_size
            resize_factor = 1.0 * np.asarray(resample_size) / np.asarray(origin_shape)
            new_array = zoom(array, resize_factor, order=1) #order = 1：bilinear interpolaion
            new_array = new_array.astype(np.float32)
            print(patient,new_array.shape,new_spacing)

        if patient in ['1', '5', '8', '10', '13', '20', '22', '31', '34', '38']: # random select validation set
            npy_save_path = op.join(save_path,'valid', str(patient))
            create_path([npy_save_path]) 
            np.save(npy_save_path+'/image.npy',new_array)
            np.save(npy_save_path+'/seg.npy',new_label)
            np.save(npy_save_path+'/spacing.npy',new_spacing)
        else :
            npy_save_path = op.join(save_path,'train', str(patient))
            create_path([npy_save_path])    
            np.save(npy_save_path+'/image.npy',new_array)
            np.save(npy_save_path+'/spacing.npy',new_spacing)
            if j == 0 :
                np.save(npy_save_path+'/seg.npy',new_label)
        print()

['1', '10', '13', '15', '19', '2', '20', '21', '22', '3', '31', '32', '33', '34', '36', '37', '38', '39', '5', '8'] 20
(256, 256, 36) (1.54296875, 1.54296875, 7.699999999999914)
uint16


['IMG-0002-00001.png', 'IMG-0002-00002.png', 'IMG-0002-00003.png', 'IMG-0002-00004.png', 'IMG-0002-00005.png', 'IMG-0002-00006.png', 'IMG-0002-00007.png', 'IMG-0002-00008.png', 'IMG-0002-00009.png', 'IMG-0002-00010.png', 'IMG-0002-00011.png', 'IMG-0002-00012.png', 'IMG-0002-00013.png', 'IMG-0002-00014.png', 'IMG-0002-00015.png', 'IMG-0002-00016.png', 'IMG-0002-00017.png', 'IMG-0002-00018.png', 'IMG-0002-00019.png', 'IMG-0002-00020.png', 'IMG-0002-00021.png', 'IMG-0002-00022.png', 'IMG-0002-00023.png', 'IMG-0002-00024.png', 'IMG-0002-00025.png', 'IMG-0002-00026.png', 'IMG-0002-00027.png', 'IMG-0002-00028.png', 'IMG-0002-00029.png', 'IMG-0002-00030.png', 'IMG-0002-00031.png', 'IMG-0002-00032.png', 'IMG-0002-00033.png', 'IMG-0002-00034.png', 'IMG-0002-00035.png', 'IMG-0002-00036.png']
label (256, 256, 36)
int8
label_unique [0 1 2 3 6]
1 (256, 256, 32) (256, 256, 32) [1.54296875 1.54296875 8.6625    ]
uint16 ggadgadgadsg

(256, 256, 36) (1.69921875, 1.69921875, 7.6999997820171435)
uint16
[